# Hyperparameter Tuning using HyperDrive

In the cell below, we import all the dependencies that we need to complete the project.

In [3]:
import joblib
import uuid
import requests
import json

from azureml.core import (
    Workspace,
    Experiment,
    Dataset,
    ComputeTarget,
    ScriptRunConfig,
    Environment
)

from azureml.train.hyperdrive import (
    BanditPolicy, 
    RandomParameterSampling,
    choice, 
    loguniform, 
    HyperDriveConfig, 
    PrimaryMetricGoal
)

from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

ModuleNotFoundError: No module named 'azureml.train'

## Workspace

In [ ]:
workspace = Workspace.from_config()

In [ ]:
print("Subscription ID:", workspace.subscription_id)
print("Resource group:", workspace.resource_group)
print("Workspace name:", workspace.name)

Subscription ID: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-239589
Workspace name: quick-starts-ws-239589


## Experiment

In [ ]:
experiment_name = 'edu_hf_hyperdrive_exp'
experiment = Experiment(workspace, experiment_name)

## Compute target

We assume a compute cluster with the given name has already been created.

In [ ]:
compute_cluster_name = "edu-compute-cluster"
compute_target = workspace.compute_targets[compute_cluster_name]

## Dataset

We use the [heart failure dataset](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data) from Kaggle.
We assume it has already been registered as an Azure ML dataset.

In [ ]:
dataset_name = 'edu_heart_failure_dataset'
dataset = Dataset.get_by_name(workspace, name=dataset_name)

In [ ]:
# Make a dataframe and take a look at it
patients = dataset.to_pandas_dataframe()
patients.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Hyperdrive Configuration

We're using a random forest (RF) classifier, because RF tend to generate reasonable predictions across a wide range of data while requiring little configuration.

We're letting HyperDrive select the best combination of the hyperparameters `n_estimators`, the number of trees in the forest, and `min_samples_split`, the minimum fraction of samples required to split an internal node.

We're using a Bandit early termination policy, which ends runs when the primary metric isn't within the specified slack factor of the most successful run.

Our primary metric is mean accuracy, which training should maximize.

Useful tutorial here: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters?view=azureml-api-1

In [ ]:
primary_metric_name = "mean accuracy"

venv = Environment.from_pip_requirements(name="venv", file_path="requirements.txt")

train_cfg = ScriptRunConfig(
    source_directory="steps",
    script="train.py",
    environment=venv,
    compute_target=compute_target,
)

param_sampling = RandomParameterSampling({
    "n_estimators": choice(20, 50, 100, 200),
    "min_samples_split": loguniform(-6, -2),
})

early_termination_policy = BanditPolicy(slack_factor=0.2)

hyperdrive_run_config = HyperDriveConfig(
    run_config=train_cfg,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name=primary_metric_name,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=4
)

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, we use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

In the cells below, we get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(f"Best run id: {best_run.id}")

Best run id: HD_3d583775-50d1-48cc-b95b-db291bdd2bbe_23


In [ ]:
best_run_metrics = best_run.get_metrics()
for key, value in best_run_metrics.items():
    print(f"{key}: {value}")

Number of trees in the forest: 200
Minimum fraction of samples required to split an internal node: 0.040621547359347976
mean accuracy: 0.92


In [ ]:
best_run.get_file_names()

['outputs/model.joblib',
 'outputs/scaler.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [ ]:
best_run.download_files()

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Useful tutorial here: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?view=azureml-api-1&tabs=python

In [ ]:
model_name = "RF_best_model"
model = best_run.register_model(model_name=model_name, model_path="outputs")

In [ ]:
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
)

inference_config = InferenceConfig(
    entry_script='score.py'
    environment=venv,
)

service = Model.deploy(
    workspace,
    "edu-service",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
uri = service.scoring_uri

headers = {"Content-Type": "application/json"}

target_column = "DEATH_EVENT"
json_payload = {
    "data": patients.drop(columns=target_column).sample(n=2).to_dict("records")
}
raw_data = json.dumps(json_payload)

response = requests.post(uri, data=raw_data, headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()
model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

